In [2]:
from montecarlo import MonteCarlo
from scipy.stats import norm
import numpy as np

Analytical value of $\delta$: 

In [2]:
norm.cdf(1 / (0.2 * np.sqrt(1)) * (np.log(100 / 99) + (0.06 + ((0.2 ** 2) / 2)) * 1)) - 1

-0.3262644882651039

## Bump and evaluate
Different seed:

In [14]:
n = 100
for s in [100, 1000, 10000]:
    v_bumped = np.empty((n, s))
    v_unbumped = np.empty((n, s))
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v_bumped[i] = MonteCarlo(100+e, 0.06, 1, 99, 365, 0.2).run_immediately(s)
            v_unbumped[i] = MonteCarlo(100, 0.06, 1, 99, 365, 0.2).run_immediately(s)
        deltas = (np.mean(v_bumped, axis=1) - np.mean(v_unbumped, axis=1)) / e
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas, ddof=1) / np.sqrt(n)})')

e=1, s=100: delta=-0.3146674777432673 (0.1101716369572514)
e=0.1, s=100: delta=-0.7772623816644831 (1.0994324104756423)
e=0.01, s=100: delta=11.477743673070076 (11.254904655295336)
e=1, s=1000: delta=-0.2940283892351972 (0.0364882525466816)
e=0.1, s=1000: delta=-0.2740658803227782 (0.3782688066347183)
e=0.01, s=1000: delta=3.0329767832300676 (3.76294374308032)
e=1, s=10000: delta=-0.31529133360563644 (0.010263043150406048)
e=0.1, s=10000: delta=-0.3867792956471142 (0.11875059248534167)
e=0.01, s=10000: delta=-0.022891024078042967 (1.1069021350872994)


Same seed:

In [15]:
n = 100
for s in [100, 1000, 10000]:
    v_bumped = np.empty((n, s))
    v_unbumped = np.empty((n, s))
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v_bumped[i] = MonteCarlo(100+e, 0.06, 1, 99, 365, 0.2).run_immediately(s, seed=i)
            v_unbumped[i] = MonteCarlo(100, 0.06, 1, 99, 365, 0.2).run_immediately(s, seed=i)
        deltas = (np.mean(v_bumped, axis=1) - np.mean(v_unbumped, axis=1)) / e
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas, ddof=1) / np.sqrt(n)})')

e=1, s=100: delta=-0.31700541743583693 (0.0038280956657927184)
e=0.1, s=100: delta=-0.3241439763237034 (0.0038114159713037534)
e=0.01, s=100: delta=-0.3248974235110383 (0.003787338253978304)
e=1, s=1000: delta=-0.31715142278333147 (0.0012415164864747377)
e=0.1, s=1000: delta=-0.3253099948341837 (0.0012649022082270587)
e=0.01, s=1000: delta=-0.3261716624976118 (0.001265882525481593)
e=1, s=10000: delta=-0.3171257583332522 (0.0003860264322810956)
e=0.1, s=10000: delta=-0.32509618464485385 (0.0003954455721241265)
e=0.01, s=10000: delta=-0.3259315658377204 (0.00039218195568049797)


## Delta digital option
Using bump and evaluate:

In [16]:
 n = 100
for s in [100, 1000, 10000]:
    v_bumped = np.empty((n, s))
    v_unbumped = np.empty((n, s))
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v_bumped[i] = MonteCarlo(100+e, 0.06, 1, 99, 365, 0.2, digital=True).run_immediately(s, seed=i)
            v_unbumped[i] = MonteCarlo(100, 0.06, 1, 99, 365, 0.2, digital=True).run_immediately(s, seed=i)
        deltas = (np.mean(v_bumped, axis=1) - np.mean(v_unbumped, axis=1)) / e
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas, ddof=1) / np.sqrt(n)})')

e=1, s=100: delta=-0.016009997070932223 (0.001178394163665624)
e=0.1, s=100: delta=-0.01695176160451647 (0.003636368713448445)
e=0.01, s=100: delta=-0.02825293600752743 (0.01614623348156168)
e=1, s=1000: delta=-0.018223143724855215 (0.0003736912827743004)
e=0.1, s=1000: delta=-0.01921199648511868 (0.001211531480732322)
e=0.01, s=1000: delta=-0.01789352613810047 (0.004377540029338874)
e=1, s=10000: delta=-0.017864331437559614 (0.00012732354079974774)
e=0.1, s=10000: delta=-0.018694025991647358 (0.00038711522578192207)
e=0.01, s=10000: delta=-0.01723429096459156 (0.001128332385577078)


Using Likelihood ratio:

In [17]:
n = 100
for s in [100, 1000, 10000]:
    v = np.empty((n, s))
    deltas = np.empty(n)
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v[i] = MonteCarlo(100, 0.06, 1, 99, 365, 0.2, digital=True).run_immediately(s, seed=i)
            
            np.random.seed(i)
            epsilons = np.random.normal(size=s)
            deltas[i] = np.mean(v[i] * epsilons / (100 * 0.2 * np.sqrt(1)))
        
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas, ddof=1) / np.sqrt(n)})')

e=1, s=100: delta=-0.018085876561628328 (0.0002705895709301)
e=0.1, s=100: delta=-0.018085876561628328 (0.0002705895709301)
e=0.01, s=100: delta=-0.018085876561628328 (0.0002705895709301)
e=1, s=1000: delta=-0.01815522588671591 (8.77926978222565e-05)
e=0.1, s=1000: delta=-0.01815522588671591 (8.77926978222565e-05)
e=0.01, s=1000: delta=-0.01815522588671591 (8.77926978222565e-05)
e=1, s=10000: delta=-0.018200035014475827 (2.607577012394789e-05)
e=0.1, s=10000: delta=-0.018200035014475827 (2.607577012394789e-05)
e=0.01, s=10000: delta=-0.018200035014475827 (2.607577012394789e-05)


Using pathwise:

In [18]:
n = 100
for s in [100, 1000, 10000]:
    v = np.empty((n, s))
    deltas = np.empty(n)
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            stock_prices = np.array([100] * s, dtype=np.float64)
            stock_prices *= np.exp((0.06 - 0.5 * 0.2**2) + 0.2 *  np.random.normal(size=s))
            payoffs = norm.pdf(stock_prices, 99) * np.exp(-0.06)

            deltas[i] = np.mean(payoffs * stock_prices / 100)
        
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas, ddof=1) / np.sqrt(n)})')

e=1, s=100: delta=0.018956056087610515 (0.0006757135134241316)
e=0.1, s=100: delta=0.01725403211426736 (0.0007364437321554505)
e=0.01, s=100: delta=0.017500785328692745 (0.0006062587348751807)
e=1, s=1000: delta=0.01823847349646597 (0.00021439862674964364)
e=0.1, s=1000: delta=0.018014408540463343 (0.0002192774211176674)
e=0.01, s=1000: delta=0.018491521264774838 (0.00023743120459168002)
e=1, s=10000: delta=0.018200700345436263 (6.389803363131993e-05)
e=0.1, s=10000: delta=0.01818272669249265 (6.904297019331926e-05)
e=0.01, s=10000: delta=0.018248780814510722 (7.122910945455674e-05)
